In [28]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt 
import pandas as pd

from sklearn.datasets import load_iris
import numpy as np

In [2]:
torch.manual_seed(2)

# Create Pandas Dataframe with multiple y-classes

In [100]:
iris = load_iris()
X = iris.data
Y = iris.target

zufällig 30 X-Werte mit den dazugehörigen y-Werten ziehen für die Prediction (Art Validation Set)

In [101]:
from sklearn.model_selection import train_test_split
x, x_val, y, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

Split y Variable and x Variables

In [102]:
x.shape, y.shape, x_val.shape, y_val.shape

((100, 4), (100,), (50, 4), (50,))

Define each column as a Pytorch Tensor

In [103]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

In [104]:
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

In [105]:
x_train.shape

(100, 4)

Put Data through DataLoader, so we can use batches

In [106]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x_train)
        self.y=torch.from_numpy(y_train)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [107]:
data_set=Data()

In [108]:
trainloader=DataLoader(dataset=data_set,batch_size=64)

In [109]:
data_set.x

tensor([[5.7000, 2.9000, 4.2000, 1.3000],
        [7.6000, 3.0000, 6.6000, 2.1000],
        [5.6000, 3.0000, 4.5000, 1.5000],
        [5.1000, 3.5000, 1.4000, 0.2000],
        [7.7000, 2.8000, 6.7000, 2.0000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [5.0000, 3.5000, 1.3000, 0.3000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [5.0000, 2.0000, 3.5000, 1.0000],
        [6.3000, 2.7000, 4.9000, 1.8000],
        [4.8000, 3.4000, 1.9000, 0.2000],
        [5.0000, 3.0000, 1.6000, 0.2000],
        [5.1000, 3.3000, 1.7000, 0.5000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [5.1000, 3.4000, 1.5000, 0.2000],
        [5.7000, 3.0000, 4.2000, 1.2000],
        [7.7000, 3.8000, 6.7000, 2.2000],
        [4.6000, 3.2000, 1.4000, 0.2000],
        [6.2000, 2.9000, 4.3000, 1.3000],
        [5.7000, 2.5000, 5.0000, 2.0000],
        [5.5000, 4.2000, 1.4000, 0.2000],
        [6.0000, 3.0000, 4.8000, 1.8000],
        [5.8000, 2.7000, 5.1000, 1

In [110]:
data_set.y

tensor([1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2,
        1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0,
        2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0,
        1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1,
        2, 0, 1, 2])

In [111]:
data_set.x.shape, data_set.y.shape

(torch.Size([100, 4]), torch.Size([100]))

# Build Model and train it

Build softmax Classifier

In [112]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [113]:
input_dim=4     # wie viele Variablen haben wir pro Zeile
hidden_dim = 25 # Anzahl hidden layers
output_dim=3    # wie viele Klassen wollen wir klassifizieren
input_dim

4

In [114]:
# Wir geben dem Modell mit, wie viele X-Variablen wir haben (hier 3) und wie viele y-Klassen wir haben (hier 3)
model=Net(input_dim,hidden_dim,output_dim)

View the size of the model parameters:

In [115]:
print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

W: torch.Size([25, 4])
b torch.Size([25])


Loss function

In [116]:
criterion=nn.CrossEntropyLoss()

Optimizer

In [117]:
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [118]:
n_epochs=1000
loss_list=[]

#n_epochs
for epoch in range(n_epochs):
    for x, y in trainloader:
      

        #clear gradient 
        optimizer.zero_grad()
        #make a prediction 
        z=model(x)
        # calculate loss, da Cross Entropy benutzt wird muss ich in den loss Klassen vorhersagen, 
        # also Wahrscheinlichkeit pro Klasse. Das mach torch.max(y,1)[1])
        loss=criterion(z,y)
        # calculate gradients of parameters 
        loss.backward()
        # update parameters 
        optimizer.step()
        
        loss_list.append(loss.data)
        
        
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 1.0814207792282104
epoch 0, loss 1.0785229206085205
epoch 1, loss 1.0672663450241089
epoch 1, loss 1.0540757179260254
epoch 2, loss 1.0559152364730835
epoch 2, loss 1.0371512174606323
epoch 3, loss 1.0440027713775635
epoch 3, loss 1.023028016090393
epoch 4, loss 1.0310920476913452
epoch 4, loss 1.0099791288375854
epoch 5, loss 1.0174051523208618
epoch 5, loss 0.9973725080490112
epoch 6, loss 1.0032309293746948
epoch 6, loss 0.9850056171417236
epoch 7, loss 0.9888116717338562
epoch 7, loss 0.9728457927703857
epoch 8, loss 0.9743290543556213
epoch 8, loss 0.9609141945838928
epoch 9, loss 0.9599066972732544
epoch 9, loss 0.9492301940917969
epoch 10, loss 0.9456192851066589
epoch 10, loss 0.9377956390380859
epoch 11, loss 0.9315028786659241
epoch 11, loss 0.9265930652618408
epoch 12, loss 0.9175696969032288
epoch 12, loss 0.9155962467193604
epoch 13, loss 0.9038184285163879
epoch 13, loss 0.9047762751579285
epoch 14, loss 0.8902431726455688
epoch 14, loss 0.8941075801849365
e

epoch 162, loss 0.3248670697212219
epoch 163, loss 0.30522027611732483
epoch 163, loss 0.32300513982772827
epoch 164, loss 0.3038816452026367
epoch 164, loss 0.3211544454097748
epoch 165, loss 0.30255234241485596
epoch 165, loss 0.31931501626968384
epoch 166, loss 0.3012322783470154
epoch 166, loss 0.31748682260513306
epoch 167, loss 0.299921452999115
epoch 167, loss 0.31566983461380005
epoch 168, loss 0.29861992597579956
epoch 168, loss 0.31386417150497437
epoch 169, loss 0.29732760787010193
epoch 169, loss 0.31206974387168884
epoch 170, loss 0.2960445284843445
epoch 170, loss 0.31028664112091064
epoch 171, loss 0.2947705388069153
epoch 171, loss 0.3085150122642517
epoch 172, loss 0.2935057580471039
epoch 172, loss 0.30675458908081055
epoch 173, loss 0.2922500967979431
epoch 173, loss 0.3050055503845215
epoch 174, loss 0.2910037040710449
epoch 174, loss 0.30326783657073975
epoch 175, loss 0.2897662818431854
epoch 175, loss 0.3015414774417877
epoch 176, loss 0.2885378897190094
epoch 17

epoch 332, loss 0.14251594245433807
epoch 333, loss 0.17820638418197632
epoch 333, loss 0.1419890969991684
epoch 334, loss 0.17784225940704346
epoch 334, loss 0.14146581292152405
epoch 335, loss 0.17748065292835236
epoch 335, loss 0.14094625413417816
epoch 336, loss 0.1771213263273239
epoch 336, loss 0.14043028652668
epoch 337, loss 0.17676451802253723
epoch 337, loss 0.13991791009902954
epoch 338, loss 0.17641009390354156
epoch 338, loss 0.13940909504890442
epoch 339, loss 0.17605805397033691
epoch 339, loss 0.13890384137630463
epoch 340, loss 0.1757083535194397
epoch 340, loss 0.13840201497077942
epoch 341, loss 0.1753610372543335
epoch 341, loss 0.13790366053581238
epoch 342, loss 0.17501597106456757
epoch 342, loss 0.1374087929725647
epoch 343, loss 0.1746731996536255
epoch 343, loss 0.13691727817058563
epoch 344, loss 0.17433269321918488
epoch 344, loss 0.136429101228714
epoch 345, loss 0.1739945113658905
epoch 345, loss 0.13594430685043335
epoch 346, loss 0.17365841567516327
epoc

epoch 503, loss 0.1393137276172638
epoch 503, loss 0.08724907040596008
epoch 504, loss 0.13917236030101776
epoch 504, loss 0.08705639094114304
epoch 505, loss 0.1390315741300583
epoch 505, loss 0.08686469495296478
epoch 506, loss 0.13889145851135254
epoch 506, loss 0.08667393773794174
epoch 507, loss 0.1387520283460617
epoch 507, loss 0.08648405224084854
epoch 508, loss 0.13861316442489624
epoch 508, loss 0.08629505336284637
epoch 509, loss 0.1384749412536621
epoch 509, loss 0.08610700070858002
epoch 510, loss 0.13833734393119812
epoch 510, loss 0.08591989427804947
epoch 511, loss 0.13820037245750427
epoch 511, loss 0.08573362976312637
epoch 512, loss 0.1380639672279358
epoch 512, loss 0.0855482891201973
epoch 513, loss 0.13792820274829865
epoch 513, loss 0.08536384999752045
epoch 514, loss 0.13779307901859283
epoch 514, loss 0.08518024533987045
epoch 515, loss 0.13765849173069
epoch 515, loss 0.0849975124001503
epoch 516, loss 0.1375245600938797
epoch 516, loss 0.08481571078300476
epo

epoch 678, loss 0.06365539878606796
epoch 679, loss 0.12153900414705276
epoch 679, loss 0.06356190145015717
epoch 680, loss 0.12146728485822678
epoch 680, loss 0.0634685754776001
epoch 681, loss 0.1213957890868187
epoch 681, loss 0.06337563693523407
epoch 682, loss 0.1213245838880539
epoch 682, loss 0.0632830560207367
epoch 683, loss 0.12125357985496521
epoch 683, loss 0.0631907656788826
epoch 684, loss 0.12118276208639145
epoch 684, loss 0.06309883296489716
epoch 685, loss 0.12111223489046097
epoch 685, loss 0.06300715357065201
epoch 686, loss 0.12104190140962601
epoch 686, loss 0.06291583180427551
epoch 687, loss 0.12097180634737015
epoch 687, loss 0.0628248080611229
epoch 688, loss 0.1209019273519516
epoch 688, loss 0.06273413449525833
epoch 689, loss 0.12083230167627335
epoch 689, loss 0.06264374405145645
epoch 690, loss 0.1207628846168518
epoch 690, loss 0.06255371123552322
epoch 691, loss 0.12069369107484818
epoch 691, loss 0.0624639131128788
epoch 692, loss 0.12062469869852066
e

epoch 855, loss 0.0510113388299942
epoch 856, loss 0.11170312017202377
epoch 856, loss 0.05095700919628143
epoch 857, loss 0.11166028678417206
epoch 857, loss 0.05090288445353508
epoch 858, loss 0.11161749064922333
epoch 858, loss 0.05084885656833649
epoch 859, loss 0.11157477647066116
epoch 859, loss 0.050795018672943115
epoch 860, loss 0.11153221875429153
epoch 860, loss 0.0507412888109684
epoch 861, loss 0.11148971319198608
epoch 861, loss 0.05068783089518547
epoch 862, loss 0.11144740134477615
epoch 862, loss 0.05063430964946747
epoch 863, loss 0.11140511929988861
epoch 863, loss 0.050581131130456924
epoch 864, loss 0.11136297881603241
epoch 864, loss 0.05052797123789787
epoch 865, loss 0.11132097244262695
epoch 865, loss 0.050474993884563446
epoch 866, loss 0.11127904802560806
epoch 866, loss 0.05042213201522827
epoch 867, loss 0.11123725771903992
epoch 867, loss 0.05036944895982742
epoch 868, loss 0.11119550466537476
epoch 868, loss 0.050316862761974335
epoch 869, loss 0.11115388

In [119]:
z

tensor([[  8.1887,   2.9522, -11.6765],
        [ -5.7342,   1.8318,   3.5589],
        [ -1.8169,   3.2295,  -1.9037],
        [ -5.6675,   1.8034,   3.5129],
        [  0.5174,   3.9692,  -5.0287],
        [ -2.3082,   2.9022,  -1.0718],
        [ -2.6742,   3.0831,  -0.8883],
        [  6.6503,   3.3501, -10.5125],
        [ -1.9534,   3.5696,  -2.1054],
        [ -3.3759,   2.7797,   0.1353],
        [  8.2457,   2.9730, -11.7482],
        [ -1.3329,   3.3340,  -2.5216],
        [ -7.0737,   1.3762,   5.4000],
        [ -6.3565,   1.4792,   4.5404],
        [  8.1223,   2.9590, -11.5950],
        [ -0.2339,   3.5658,  -3.8621],
        [ -7.0558,   1.1002,   5.6628],
        [ -5.4413,   2.1941,   2.8816],
        [  7.6744,   3.1161, -11.3261],
        [ -6.9955,   1.1709,   5.5334],
        [  7.6464,   3.1249, -11.2932],
        [ -2.5477,   3.3517,  -1.2726],
        [ -6.9074,   1.4752,   5.1397],
        [ -6.6427,   1.3997,   4.9134],
        [ -1.7447,   3.3122,  -2.0669],


In [120]:
yhat=torch.max(z.data,1)

In [121]:
yhat

(tensor([8.1887, 3.5589, 3.2295, 3.5129, 3.9692, 2.9022, 3.0831, 6.6503, 3.5696,
         2.7797, 8.2457, 3.3340, 5.4000, 4.5404, 8.1223, 3.5658, 5.6628, 2.8816,
         7.6744, 5.5334, 7.6464, 3.3517, 5.1397, 4.9134, 3.3122, 3.9547, 3.1473,
         3.7061, 4.6432, 3.6151, 7.7522, 3.7042, 4.2777, 8.7158, 3.5086, 5.1920]),
 tensor([0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2,
         1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2]))

Erklärung zu z.data: er geht über jede Zeile, zu welchem Wert der drei Ausprägungen die Zeile gehören kann. Daher pro Zeile 3 Werte. Spalte 1 steht für Klasse 0, Spalte 2 steht für Klasse 1 und Spalte 3 steht für Klasse 2. Jetzt wählt er aus jeder Zeile den höchsten Wert und guckt in welcher Spalte sich dieser Wert befindet. Die Klasse der Spalte wird dann als Klasse für die Zeile genommen. 

In [122]:
y

tensor([0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2,
        1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

Alle Klassen werden korrekt predicted.

Auf Validation Set

In [123]:
z=model(x_val)

In [124]:
yhat=torch.max(z.data,1)
yhat

(tensor([2.9331, 8.1057, 7.5182, 2.7353, 3.1439, 7.8844, 3.5647, 3.5067, 2.3791,
         3.5174, 2.9054, 7.6687, 8.3063, 7.6536, 8.2286, 2.8957, 6.2463, 3.3859,
         2.6780, 6.2222, 7.5084, 2.7361, 7.6717, 6.0160, 3.5364, 4.5461, 5.9379,
         6.0292, 7.5373, 7.4079, 8.5239, 8.6176, 3.6727, 7.7552, 7.8798, 4.4914,
         3.2854, 8.0514, 8.2126, 8.5125, 5.1841, 2.9305, 3.2502, 8.4736, 8.2424,
         3.5674, 2.5656, 4.2622, 3.5569, 5.7832]),
 tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
         2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
         1, 2]))

In [125]:
y_val

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2])